In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout



In [2]:
data = pd.read_csv('content/final_data.csv')

In [3]:
import pandas as pd

# 假設 data_sorted 是你已經排序好的資料
data_sorted = data.copy()

# 獲取所有數值型特徵列，排除 'playerid' 和 'gameid'
columns_to_exclude = ['playerid', 'gameid']
features_to_replace = data_sorted.select_dtypes(include=['float64', 'int64']).columns
features_to_replace = [col for col in features_to_replace if col not in columns_to_exclude]

# 定義前幾場比賽的數據平均值，這個變量可以調整
num_games_to_average = 10  # 可以調整這個值，表示取前幾場的數據平均

# 從最後一行開始，往前尋找對應的 playerid 的前幾場比賽
for i in range(len(data_sorted) - 1, -1, -1):
    current_player = data_sorted.loc[i, 'playerid']

    # 保存找到的前幾場比賽數據的列表
    previous_games = []

    # 遍歷查找前幾場比賽
    for j in range(i - 1, -1, -1):
        if data_sorted.loc[j, 'playerid'] == current_player and data_sorted.loc[j, 'SEC'] > 0:
            previous_games.append(data_sorted.loc[j, features_to_replace])
        if len(previous_games) >= num_games_to_average:
            break

    # 如果找到的前幾場比賽數據不為空，計算平均值
    if len(previous_games) > 0:
        average_values = pd.DataFrame(previous_games).mean()
        data_sorted.loc[i, features_to_replace] = average_values
    else:
        # 如果沒有找到前幾場比賽，將當前行數據設為 0
        data_sorted.loc[i, features_to_replace] = 0

# 查看結果
data_sorted.head()


C:\Users\barry\AppData\Local\Temp\ipykernel_13092\122057332.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '32.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_sorted.loc[i, features_to_replace] = average_values
C:\Users\barry\AppData\Local\Temp\ipykernel_13092\122057332.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_sorted.loc[i, features_to_replace] = average_values
C:\Users\barry\AppData\Local\Temp\ipykernel_13092\122057332.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_sorted.loc

,gameid,playerid,estOFFRTG,OFFRTG,estDEFRTG,DEFRTG,estNETRTG,NETRTG,ASTpct,ASTtoTOV,...,passes,contestedFGM,contestedFGA,contestedFGpct,uncontestedFGM,uncontestedFGA,uncontestedFGpct,defendedatrimFGM,defendedatrimFGA,defendedatrimFGpct
0,22200001,1627759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22200001,1628369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22200001,201143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22200001,1628401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22200001,203935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# import pandas as pd

# # 假設 data_sorted 是你已經排序好的資料
# data_sorted = data.copy()

# # 獲取所有數值型特徵列，排除 'playerid'、'gameid' 和 'SEC'
# columns_to_exclude = ['playerid', 'gameid', 'SEC']
# features_to_replace = data_sorted.select_dtypes(include=['float64', 'int64']).columns
# features_to_replace = [col for col in features_to_replace if col not in columns_to_exclude]

# # 定義前幾場比賽的數據平均值，這個變量可以調整
# num_games_to_average = 3  # 可以調整這個值，表示取前幾場的數據平均

# # 使用 groupby 分組，避免逐行遍歷
# grouped = data_sorted.groupby('playerid')

# # 定義一個函數來處理每個分組
# def process_group(group):
#     # 遍歷每個 group，找出 SEC <= 0 的比賽，並用前幾場 SEC > 10 的數據進行填充
#     for i in range(len(group) - 1, -1, -1):
#         if group.iloc[i]['SEC'] <= 0:
#             # 選取前幾場比賽 SEC > 10 的數據
#             previous_games = group.iloc[:i][group.iloc[:i]['SEC'] > 10].tail(num_games_to_average)
            
#             # 如果找到了符合條件的比賽數據，計算平均
#             if len(previous_games) > 0:
#                 group.iloc[i, group.columns.get_loc(features_to_replace)] = previous_games[features_to_replace].mean().values
#             else:
#                 # 如果沒有找到，將特徵值設為 0
#                 group.iloc[i, group.columns.get_loc(features_to_replace)] = 0
    
#     return group

# # 對每個 playerid 的分組應用該函數
# data_sorted = grouped.apply(process_group)

# # 查看結果
# data_sorted.head()


In [5]:
grouped_data = pd.DataFrame(data_sorted)
grouped_data.to_csv('content/LookingFoward_data_avg_d2_3.csv', index=False)

In [6]:
data = pd.read_csv('content/LookingFoward_data_avg_d2_3.csv')

In [7]:
data.head()

,gameid,playerid,estOFFRTG,OFFRTG,estDEFRTG,DEFRTG,estNETRTG,NETRTG,ASTpct,ASTtoTOV,...,passes,contestedFGM,contestedFGA,contestedFGpct,uncontestedFGM,uncontestedFGA,uncontestedFGpct,defendedatrimFGM,defendedatrimFGA,defendedatrimFGpct
0,22200001,1627759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22200001,1628369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22200001,201143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22200001,1628401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22200001,203935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# grouped_data

In [9]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
# 標準化
scaler = MinMaxScaler()

# 主客場
data['home_basic'] = data['home_basic'].apply(lambda x: 1 if x == 'TRUE' else 0)
# 假设 'win' 列包含 'TRUE' 和 'FALSE' 字符串
data['win'] = data['win'].map({'TRUE': 1, 'FALSE': 0})

# 如果你要对所有包含 'TRUE' 和 'FALSE' 的列进行转换
bool_columns = data.columns[data.isin(['TRUE', 'FALSE']).any()]

# 对所有这些列进行转换
for col in bool_columns:
    data[col] = data[col].map({'TRUE': 1, 'FALSE': 0})

In [10]:
import pandas as pd

def filter_games_by_players(data, player_ids):
    # 找出這些 player 參與的所有比賽
    player_games = data[data['playerid'].isin(player_ids)]['gameid'].unique()

    # 初始化結果存儲
    result = []

    # 檢查每一場比賽，並確保不重複篩選其他已在同場比賽中的 player
    for game in player_games:
        # 確認這場比賽中的所有 player
        players_in_game = data[data['gameid'] == game]['playerid'].unique()
        result.append({
            'gameid': game,
            'players': list(players_in_game)
        })

    return result

def get_filtered_game_data(data, player_ids):
    # 使用上述 function 找出 players 參與的比賽
    game_info = filter_games_by_players(data, player_ids)

    # 找出這些比賽的所有資料
    game_ids = [game['gameid'] for game in game_info]

    # 根據 gameid 篩選出所有相關資料
    filtered_data = data[data['gameid'].isin(game_ids)]

    return filtered_data

# 載入數據
file_path = 'path_to_your_file.csv'  # 替換為實際檔案路徑
# data = pd.read_csv(file_path)

# 合併 player ids
team1_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]
team2_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]

# 將兩個 list 合併
combined_player_ids = team1_player_ids + team2_player_ids

# 過濾出這些 player 的比賽資料
filtered_game_data = get_filtered_game_data(data, combined_player_ids)

# 顯示篩選後的資料
print(filtered_game_data)

filtered_game_dataCSV = pd.DataFrame(filtered_game_data)
filtered_game_dataCSV.to_csv('content/filtered_game_data.csv', index=False)

         gameid  playerid  estOFFRTG  OFFRTG  estDEFRTG  DEFRTG  estNETRTG  \
0      22200001   1627759       0.00    0.00       0.00    0.00       0.00   
1      22200001   1628369       0.00    0.00       0.00    0.00       0.00   
2      22200001    201143       0.00    0.00       0.00    0.00       0.00   
3      22200001   1628401       0.00    0.00       0.00    0.00       0.00   
4      22200001    203935       0.00    0.00       0.00    0.00       0.00   
...         ...       ...        ...     ...        ...     ...        ...   
39595  52200211   1630544      97.70  100.73     113.95  110.20     -16.23   
39596  52200211   1630526      98.68  102.45     111.67  107.83     -12.99   
39597  52200211         0       0.00    0.00       0.00    0.00       0.00   
39598  52200211         0       0.00    0.00       0.00    0.00       0.00   
39599  52200211         0       0.00    0.00       0.00    0.00       0.00   

       NETRTG  ASTpct  ASTtoTOV  ...  passes  contestedFGM  con

In [11]:
file_path = 'content/selected_columns_s.txt'

with open(file_path, 'r') as file:
    selected_columns = [line.strip() for line in file]

print(selected_columns)

['plusminusPTS', 'estNETRTG', 'REBpct', 'estOFFRTG', 'DRB', 'FTM', 'uncontestedFGM', 'passes', 'ASTpct', 'touches', 'estUSGpct', 'oppPIP', 'TRB', 'oppFBPTS', 'PTS', 'estpace', 'FGpct', 'pie', 'DIST', 'defendedatrimFGA', 'BLK', '2ndPTS', 'TOVratio', 'pace', 'paceper40', '2ndAST', 'contestedFGA', 'OFFRTG', 'TSpct', 'FGM', 'playerid', 'team', '3Ppct', 'FTAST', 'STL', 'defendedatrimFGM', 'SEC', 'defendedatrimFGpct', 'oppPTSoffTOV', 'FBPTS', 'contestedFGM', '3PM', 'name', 'PIP', 'home', 'SPD', 'FTpct', 'PTSoffTOV', 'DEFRTG', 'contestedFGpct']


In [12]:
# 依照gameid進行分組，每一組代表一場比賽
games = data.groupby('gameid')

In [13]:
playerids = data.groupby('playerid')

In [14]:
# 將每場比賽中的所有球員數據合併成一個特徵向量
X = []
y = []

for gameid, group in games:
    if len(group) != 30:  # 每場比賽最多有30位球員
        continue

    # 將每場比賽中的所有球員數據合併成一個特徵向量
    #game_features = group.drop(columns=['gameid', 'win', 'home_basic','date']).values.flatten(
    game_features = group[selected_columns].values.flatten()
    game_features = np.append(game_features, group['date'].iloc[0])
    game_features = np.append(game_features, group['win'].iloc[0])
    X.append(game_features)
    #y.append(group['win'].iloc[0])

In [15]:
X = np.array(X)

X = pd.DataFrame(X)
#X.to_csv('/content/X.csv', index=False)
# 根據日期的值對所有行進行排序
#X_sorted = X.sort_values(by=X.columns[1501], ascending=True)
# 顯示排序後的DataFrame
#X_sorted.head()

In [16]:
df = X.reset_index(drop=True)

In [17]:
df

,0,1,2,3,4,5,6,7,8,9,...,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,5.3,5.34,0.0610,115.16,2.8,0.9,5.3,29.1,0.0930,45.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,727.3,0.0
1316,-1.5,0.59,0.0813,110.66,1.6,1.0,0.7,10.0,0.0596,15.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,838.9,1.0
1317,6.7,8.16,0.0366,116.85,2.2,4.4,5.3,24.6,0.0655,40.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,739.1,0.0
1318,-8.0,-12.82,0.0748,102.80,3.7,6.1,4.3,51.3,0.2834,77.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850.5,1.0


In [18]:
# 找到"team"的位置
team_index = [i for i, col in enumerate(selected_columns) if "team" in col]

# 輸出結果
print(team_index)

[31]


In [19]:
y = np.array(df[1501])

# 勝負
y = np.where(y == 1, 1, 0)
y = to_categorical(y, 2)

In [20]:
Y = pd.DataFrame(y)

In [21]:
Y

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
1315,1.0,0.0
1316,0.0,1.0
1317,1.0,0.0
1318,0.0,1.0


In [22]:
df.drop(df.columns[1501], axis=1, inplace=True)

In [23]:
df

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,5.3,5.34,0.0610,115.16,2.8,0.9,5.3,29.1,0.0930,45.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,727.3
1316,-1.5,0.59,0.0813,110.66,1.6,1.0,0.7,10.0,0.0596,15.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,838.9
1317,6.7,8.16,0.0366,116.85,2.2,4.4,5.3,24.6,0.0655,40.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,739.1
1318,-8.0,-12.82,0.0748,102.80,3.7,6.1,4.3,51.3,0.2834,77.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850.5


In [24]:
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3, shuffle = False)
X_train_df = pd.DataFrame(X_train)
# X_train_df.to_csv('content/X_trainA.csv', index=False)

In [25]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,7.3,10.42,0.0650,117.38,3.6,0.9,3.6,29.1,0.1030,41.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,537.5
920,-0.3,1.48,0.0578,115.84,3.0,3.1,2.5,25.7,0.1349,45.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,535.8
921,-2.6,-6.38,0.0690,110.40,2.7,1.3,1.5,36.3,0.1081,46.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,533.9
922,0.4,0.18,0.0736,118.29,3.6,2.7,3.8,19.6,0.0940,34.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,538.0


In [26]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
924,-0.3,-0.21,0.0542,110.98,2.7,1.1,2.2,23.8,0.0649,35.8,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,475.3
925,-10.6,-19.60,0.1032,109.94,4.7,3.3,1.7,29.1,0.1107,42.5,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,537.0
926,-0.9,-0.73,0.0366,105.12,1.8,1.2,2.6,20.0,0.0836,32.6,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,525.7
927,8.1,17.32,0.0818,126.80,3.3,2.6,3.6,20.5,0.0846,36.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,513.9
928,0.6,-5.18,0.0409,114.58,0.5,0.0,1.3,10.4,0.1403,13.9,...,0.285714,96.0,5.428571,12.571429,3.961429,0.357143,0.571429,118.2,0.545857,371.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,5.3,5.34,0.0610,115.16,2.8,0.9,5.3,29.1,0.0930,45.1,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,727.3
1316,-1.5,0.59,0.0813,110.66,1.6,1.0,0.7,10.0,0.0596,15.4,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,838.9
1317,6.7,8.16,0.0366,116.85,2.2,4.4,5.3,24.6,0.0655,40.7,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,739.1
1318,-8.0,-12.82,0.0748,102.80,3.7,6.1,4.3,51.3,0.2834,77.7,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,850.5


In [27]:
y_train

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
919,1.0,0.0
920,0.0,1.0
921,1.0,0.0
922,1.0,0.0


In [28]:
y_test

,0,1
924,0.0,1.0
925,1.0,0.0
926,0.0,1.0
927,1.0,0.0
928,1.0,0.0
...,...,...
1315,1.0,0.0
1316,0.0,1.0
1317,1.0,0.0
1318,0.0,1.0


In [29]:


X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [30]:
print(y_train)

       0    1
0    0.0  1.0
1    0.0  1.0
2    0.0  1.0
3    1.0  0.0
4    0.0  1.0
..   ...  ...
919  1.0  0.0
920  0.0  1.0
921  1.0  0.0
922  1.0  0.0
923  1.0  0.0

[924 rows x 2 columns]


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

dropout_rate = 0.2

# Reshape the input data correctly
X_train = X_train.reshape(X_train.shape[0], 1, 1501)  # (samples, time steps, features)
X_test = X_test.reshape(X_test.shape[0], 1, 1501)    # (samples, time steps, features)

my_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(80, activation=tf.nn.tanh, return_sequences=True, input_shape=(1, 1501)), # Correct input shape
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.LSTM(60, activation=tf.nn.tanh, return_sequences=True),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.LSTM(40, activation=tf.nn.tanh),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(20, activation=tf.nn.sigmoid),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
my_model.fit(X_train, y_train, epochs=30, batch_size=32)

# 评估模型
loss, accuracy = my_model.evaluate(X_test, y_test, verbose=0)

print(f"模型准确性: {accuracy:.2f}")

c:\Users\barry\.conda\envs\nba_1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.5812 - loss: 0.7327
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5511 - loss: 0.7280
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5369 - loss: 0.7155
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5356 - loss: 0.7116
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6017 - loss: 0.6828
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5540 - loss: 0.6979
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6134 - loss: 0.6839
Epoch 8/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5540 - loss: 0.6904
Epoch 9/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5823 - loss: 0.6878
Epoch 10/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5880 - loss: 0.6824
Epoch 11/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5612 - loss: 0.6964 
Epoch 12/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6036 - l

In [32]:
X_train

array([[[0.47916667, 0.31733595, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.47916667, 0.31733595, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.47916667, 0.31733595, 0.        , ..., 0.        ,
         0.        , 0.        ]],

       ...,

       [[0.425     , 0.25484819, 0.41894353, ..., 0.        ,
         0.        , 0.99108966]],

       [[0.4875    , 0.31909892, 0.4468731 , ..., 0.        ,
         0.        , 0.99870058]],

       [[0.4625    , 0.28266405, 0.33636916, ..., 0.        ,
         0.        , 0.98366438]]])

In [33]:
y_train

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
919,1.0,0.0
920,0.0,1.0
921,1.0,0.0
922,1.0,0.0


In [34]:
X_test

array([[[0.47291667, 0.31527914, 0.32908318, ..., 0.        ,
         0.        , 0.88230926]],

       [[0.25833333, 0.12536729, 0.62659381, ..., 0.        ,
         0.        , 0.99684425]],

       [[0.46041667, 0.31018609, 0.22222222, ..., 0.        ,
         0.        , 0.97586783]],

       ...,

       [[0.61875   , 0.39725759, 0.22222222, ..., 0.        ,
         0.        , 1.37200668]],

       [[0.3125    , 0.19177277, 0.45415908, ..., 0.        ,
         0.        , 1.57880082]],

       [[0.52916667, 0.41684623, 0.27625987, ..., 0.        ,
         0.        , 1.45127158]]])

In [35]:
y_test

,0,1
924,0.0,1.0
925,1.0,0.0
926,0.0,1.0
927,1.0,0.0
928,1.0,0.0
...,...,...
1315,1.0,0.0
1316,0.0,1.0
1317,1.0,0.0
1318,0.0,1.0


In [36]:
# 保存模型到指定的文件路徑
my_model.save('my_model.h5')

# 保存模型架構到 JSON 格式
model_json = my_model.to_json()
with open('my_model.json', 'w') as json_file:
    json_file.write(model_json)

In [37]:
# 函數：處理球隊 ID 並將結果存入 result_array
def process_player_ids(player_ids, data, selected_columns):
    for player_id in player_ids:
        found = False
        for idx in range(len(data) - 1, -1, -1):  # 從最後一行開始
            if data.loc[idx, 'playerid'] == player_id:  # 找到對應的 player_id
                # 提取該行的特徵數據
                selected_data = data.loc[idx, selected_columns]
                # 存到結果陣列中
                result_array.append(selected_data)
                found = True
                break  # 找到後跳出內部循環，進行下一個 player_id 的處理

        if not found:
            # 如果沒有找到該 player_id，填充一行全為 0 的數據
            zero_data = pd.Series([0] * len(selected_columns), index=selected_columns)
            result_array.append(zero_data)

In [38]:
import pandas as pd
import numpy as np

# 創建一個存儲結果的新陣列
result_array = []
# 增加球員ID到15位
team1_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]
team2_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]



# 處理 team1_player_ids
process_player_ids(team1_player_ids, data, selected_columns)

# 處理 team2_player_ids
process_player_ids(team2_player_ids, data, selected_columns)

# 將結果轉換為 DataFrame 方便查看
result_df = pd.DataFrame(result_array)



In [39]:
result_df

,plusminusPTS,estNETRTG,REBpct,estOFFRTG,DRB,FTM,uncontestedFGM,passes,ASTpct,touches,...,contestedFGM,3PM,name,PIP,home,SPD,FTpct,PTSoffTOV,DEFRTG,contestedFGpct
38565,1.2,-0.11,0.1074,111.46,5.8,0.9,4.1,30.6,0.0875,45.0,...,2.2,1.4,500.0,8.4,12.8,4.009,0.5000,2.9,112.02,0.5434
38566,0.1,-2.55,0.0916,110.27,3.1,0.2,1.3,20.4,0.0849,26.0,...,0.2,1.1,422.0,0.8,12.8,4.014,0.1500,0.3,112.98,0.2000
38567,2.8,2.11,0.1355,113.73,8.2,8.5,5.2,38.5,0.1580,65.6,...,3.8,0.6,259.0,11.8,16.9,3.515,0.8724,3.2,111.74,0.5029
38568,3.5,2.26,0.0654,111.92,4.1,2.3,5.4,50.5,0.1059,71.5,...,2.3,3.2,521.0,8.0,12.8,4.249,0.6166,3.4,109.52,0.3440
38569,-1.0,-3.09,0.0826,109.05,5.7,4.6,4.6,60.1,0.3650,86.7,...,2.3,3.0,227.0,5.6,12.8,3.373,0.7031,1.7,113.46,0.3376
37021,4.2,3.88,0.1796,121.89,8.7,4.6,3.9,39.1,0.3132,65.1,...,6.8,0.3,172.0,19.0,18.0,3.966,0.5550,3.1,119.24,0.7397
38179,-2.6,-5.61,0.0855,113.20,3.8,3.0,4.8,51.7,0.3610,75.8,...,3.9,2.3,457.0,11.2,17.3,3.813,0.4356,3.5,118.41,0.6270
37819,1.6,3.51,0.0878,111.62,5.4,4.7,4.8,44.8,0.3521,73.8,...,3.9,1.7,200.0,13.0,12.6,4.050,0.6647,3.9,109.56,0.4759
38176,-3.3,-5.33,0.0605,107.88,2.8,0.4,1.0,20.0,0.0613,24.9,...,1.3,0.8,363.0,2.4,14.2,3.919,0.1667,0.7,114.79,0.3234
39542,-4.3,-8.32,0.1334,104.47,7.7,3.0,3.7,50.4,0.1398,72.5,...,3.3,0.0,30.0,11.4,11.0,3.874,0.6550,2.4,112.63,0.3915


In [40]:
feature_vector = result_df.values.flatten()

feature_vector = np.append(feature_vector,776)
# 將一維向量轉換為 DataFrame 的一行
flattened_df = pd.DataFrame([feature_vector])

In [41]:
flattened_df

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,1.2,-0.11,0.1074,111.46,5.8,0.9,4.1,30.6,0.0875,45.0,...,1.4,500.0,8.4,12.8,4.009,0.5,2.9,112.02,0.5434,776.0


In [42]:
flattened_df = scaler.fit_transform(flattened_df)

In [43]:
input_vector = flattened_df.reshape(1, 1, 1501)
predicted_class = my_model.predict(input_vector)

# 打印預測的類別
print("Predicted class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
Predicted class: [[0.2677149  0.73228514]]


In [44]:
# 創建一個存儲結果的新陣列
result_array = []
# 增加球員ID到15位
team1_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]
team2_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]

# 處理 team1_player_ids
process_player_ids(team1_player_ids, data, selected_columns)

# 處理 team2_player_ids
process_player_ids(team2_player_ids, data, selected_columns)


In [45]:
result_df = pd.DataFrame(result_array)
result_df

,plusminusPTS,estNETRTG,REBpct,estOFFRTG,DRB,FTM,uncontestedFGM,passes,ASTpct,touches,...,contestedFGM,3PM,name,PIP,home,SPD,FTpct,PTSoffTOV,DEFRTG,contestedFGpct
39120,0.6,-0.57,0.0766,109.31,4.1,2.3,3.6,37.2,0.1516,60.5,...,4.6,1.6,242.0,10.2,10.9,3.939,0.4375,2.4,113.03,0.5646
36564,-1.7,-10.19,0.1450,100.08,4.4,1.2,0.9,11.2,0.1228,18.5,...,1.0,0.2,40.0,2.6,19.0,4.086,0.3500,0.7,108.57,0.3600
39122,-0.6,-1.34,0.1175,108.25,5.3,0.3,1.5,35.1,0.1408,42.6,...,0.7,1.0,9.0,2.2,10.9,3.841,0.1500,1.2,110.66,0.4833
39123,-1.9,-1.97,0.0427,109.12,1.7,0.8,3.1,27.5,0.0707,37.4,...,0.8,2.7,126.0,2.4,10.9,4.066,0.2667,2.0,114.31,0.3233
39124,2.6,5.49,0.0596,113.23,3.3,3.5,3.4,42.2,0.2205,58.2,...,1.0,2.4,364.0,3.8,10.9,3.722,0.6558,2.9,109.07,0.2983
Unnamed 0,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.00,0.0000
36640,0.9,-4.77,0.0916,109.31,3.9,1.4,1.4,41.4,0.2128,52.3,...,2.2,0.8,117.0,5.2,25.6,4.198,0.5333,1.1,113.09,0.5537
Unnamed 1,0.0,0.00,0.0000,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.0,0.00,0.0000
38743,-4.7,-22.31,0.0593,100.04,0.9,1.1,0.6,10.7,0.0962,15.9,...,0.8,0.2,282.0,2.2,17.0,4.447,0.4300,0.5,122.90,0.6083
36639,-4.5,-24.28,0.0848,96.18,1.1,0.7,0.8,10.8,0.0494,14.3,...,0.4,0.4,485.0,0.8,23.0,3.992,0.3500,0.3,117.53,0.3000


In [46]:
feature_vector = result_df.values.flatten()
feature_vector = np.append(feature_vector,776)
# 將一維向量轉換為 DataFrame 的一行
flattened_df = pd.DataFrame([feature_vector])
flattened_df


,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500
0,0.6,-0.57,0.0766,109.31,4.1,2.3,3.6,37.2,0.1516,60.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,776.0


In [47]:
#標準化
flattened_df = scaler.fit_transform(flattened_df)
input_vector = flattened_df.reshape(1, 1, 1501)
predicted_class = my_model.predict(input_vector)

# 打印預測的類別
print("Predicted class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted class: [[0.2677149  0.73228514]]


data

In [48]:
games_data = pd.read_csv('/content/games.csv')


FileNotFoundError: [Errno 2] No such file or directory: '/content/games.csv'

In [ ]:
games_data

In [ ]:
final_data = pd.read_csv('content/final_data.csv')
final_data

In [ ]:
import pandas as pd

# 讀取資料
final_data = pd.read_csv('content/final_data.csv')
games_data = pd.read_csv('content/games.csv')

# 使用 merge 操作，基於 'gameid' 進行合併
merged_data = final_data.merge(games_data[['gameid', 'date']], on='gameid', how='left', suffixes=('', '_games'))

# 條件替換：當 playerid 不為 0 時，替換 final_data 中的 date
condition = final_data['playerid'] != 0
final_data.loc[condition, 'date'] = merged_data.loc[condition, 'date_games']

# 查看結果，確保數據被正確替換
print(final_data.head())

# 保存最終結果
final_data.to_csv('content/final_data_updated.csv', index=False)


In [ ]:
final_data.date

In [ ]:
import pandas as pd

# 假設 date 列已經存在於 final_data 中
# final_data = pd.read_csv('content/final_data.csv')

# 使用 pd.to_datetime 轉換日期格式，並提取日期部分
final_data['date'] = pd.to_datetime(final_data['date'], errors='coerce').dt.date

# 查看轉換結果
print(final_data['date'].head())

# 保存結果
final_data.to_csv('content/final_data_updated.csv', index=False)


In [ ]:
# 函數：處理球隊 ID 並將結果存入 result_array
def process_player_ids2(player_ids, data, selected_columns, date, result_array):
    # result_array = []  # 用來存放結果
    date = pd.to_datetime(date)  # 將輸入的日期轉換為 datetime 格式

    for player_id in player_ids:
        found_data = []  # 用來存放找到的符合條件的行
        found_count = 0  # 計算找到的符合條件的行數

        for idx in range(len(data) - 1, -1, -1):  # 從最後一行開始倒序遍歷
            # 找到符合條件的 player_id 且日期在所選日期之前
            if data.loc[idx, 'playerid'] == player_id and pd.to_datetime(data.loc[idx, 'date']) <= date:
                # 提取該行的特徵數據
                selected_data = data.loc[idx, selected_columns]
                # 將符合條件的數據添加到 found_data 列表中
                found_data.append(selected_data)
                found_count += 1

                # 如果已經找到了 5 行數據，則跳出循環
                if found_count == 5:
                    break

        # 如果找到的符合條件的行少於 5 行，則用 0 填充其餘的行
        while len(found_data) < 5:
            zero_data = pd.Series([0] * len(selected_columns), index=selected_columns)
            found_data.append(zero_data)

        # 將找到的數據存入結果陣列
        result_array.extend(found_data)

    return result_array

In [ ]:
import pandas as pd
import numpy as np

# 創建一個存儲結果的新陣列
result_arr1 = []
result_arr2 = []
# 增加球員ID到15位
team1_player_ids = [202699, 200782, 203954, 1630178, 201935, 203507, 201566, 1629630, 202694, 1628389, 1628419, 1630162, 1628378, 201152, 201571]
team2_player_ids = [1627759, 1629626, 201143, 1628401, 203935, 203518, 1626153, 1629602, 1629006, 201959, 1630242, 1627749, 1629019, 1626164, 202699]
date = '2023-06-20'


# 處理 team1_player_ids
process_player_ids2(team1_player_ids, final_data, selected_columns, date, result_arr1)

# 處理 team2_player_ids
process_player_ids2(team2_player_ids, final_data, selected_columns, date, result_arr2)

# 將結果轉換為 DataFrame 方便查看
result_df2 = pd.DataFrame(result_array2)



In [ ]:
result_df.shape

In [ ]:
result_df